In [ ]:
import torch
from PIL import Image
import numpy as np
import cv2
from sklearn.cluster import KMeans
from pathlib import Path

In [ ]:
# --------- Parámetros ---------
input_folder = Path("alas_png")       # Carpeta de entrada
output_folder = Path("particulas")    # Carpeta de salida
num_particles = 2000
resplandor_color = (0, 140, 255)      # Naranja brillante en BGR
text_label = "0.2 mm"

# Crear carpeta de salida si no existe
output_folder.mkdir(parents=True, exist_ok=True)

# --------- Verificar imágenes disponibles ---------
imgs = list(input_folder.glob("*.png"))
if len(imgs) == 0:
    raise FileNotFoundError(f"No se encontraron imágenes en {input_folder}")
print(f"Se encontraron {len(imgs)} imágenes en {input_folder}")

In [ ]:
# --------- Configuración del dispositivo ---------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------- Cargar modelo DINOv2 via torch.hub ---------
print("Cargando DINOv2...")
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14', pretrained=True)
model.eval().to(device)

In [100]:
# --------- Función para procesar cada imagen ---------
def procesar_imagen(path_img, path_out):
    print(f"Procesando {path_img.name}...")
    img = Image.open(path_img).convert("RGB")
    img_resized = img.resize((518,518))
    img_tensor = torch.tensor(np.array(img_resized)/255., dtype=torch.float32).permute(2,0,1).unsqueeze(0).to(device)

    mean = torch.tensor([0.485,0.456,0.406], device=device).view(1,3,1,1)
    std = torch.tensor([0.229,0.224,0.225], device=device).view(1,3,1,1)
    img_tensor = (img_tensor - mean)/std

    # --------- Extraer features ---------
    with torch.no_grad():
        features = model.forward_features(img_tensor)
        if 'x_norm_patchtokens' in features:
            patch_tokens = features['x_norm_patchtokens']  # [1, num_patches, dim]
        else:
            raise KeyError(f"No se encontró 'x_norm_patchtokens' en features: {features.keys()}")

    # Hacer cuadrado aproximado para reshape
    num_patches = patch_tokens.shape[1]
    dim = patch_tokens.shape[2]
    h_w = int(np.ceil(np.sqrt(num_patches)))
    pad = h_w*h_w - num_patches
    if pad > 0:
        patch_tokens = torch.cat([patch_tokens, torch.zeros((1,pad,dim), device=device)], dim=1)

    feat_tensor = patch_tokens.squeeze(0).reshape(h_w, h_w, dim)  # [H_patch, W_patch, dim]

    # --------- Segmentación con KMeans ---------
    X = feat_tensor.cpu().numpy().reshape(-1, feat_tensor.shape[-1])  # pasar a CPU para KMeans
    kmeans = KMeans(n_clusters=2, random_state=42).fit(X)
    labels = kmeans.labels_.reshape(h_w, h_w)

    # Crear máscara del ala
    ala_mask = (labels != np.bincount(labels.flatten()).argmax()).astype(np.uint8) * 255
    ala_mask = cv2.resize(ala_mask, (518,518), interpolation=cv2.INTER_NEAREST)

    # Refinar máscara
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    ala_mask = cv2.morphologyEx(ala_mask, cv2.MORPH_CLOSE, kernel)
    ala_mask = cv2.morphologyEx(ala_mask, cv2.MORPH_OPEN, kernel)

    # Detectar venas
    gray = cv2.cvtColor(np.array(img_resized), cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    edges = cv2.bitwise_and(edges, edges, mask=ala_mask)
    edges_dilated = cv2.dilate(edges, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)))

    # Fondo con partículas
    fondo = np.zeros((518,518,3), dtype=np.uint8)
    for _ in range(num_particles):
        x, y = np.random.randint(0,518), np.random.randint(0,518)
        fondo[y,x] = (200,200,200)
    for _ in range(num_particles//10):
        x, y = np.random.randint(0,518), np.random.randint(0,518)
        fondo[y,x] = (255,255,0)

    # Resplandor
    resplandor = np.zeros_like(fondo)
    resplandor_layer = cv2.bitwise_or(ala_mask, edges_dilated)
    resplandor[resplandor_layer>0] = resplandor_color
    resplandor = cv2.GaussianBlur(resplandor, (21,21), 0)

    combinado = cv2.addWeighted(fondo, 1.0, resplandor, 0.7, 0)
    ala_sobre = np.zeros_like(fondo)
    ala_sobre[ala_mask>0] = resplandor_color
    ala_sobre[edges>0] = (0,140,255)
    combinado = cv2.addWeighted(combinado, 1.0, ala_sobre, 1.0, 0)

    cv2.putText(combinado, text_label, (10, 510), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    cv2.imwrite(str(path_out), combinado)
    print(f"Guardado: {path_out}")

# --------- Procesar todas las imágenes ---------
for img_path in imgs:
    out_path = output_folder / img_path.name
    procesar_imagen(img_path, out_path)

print("¡Procesamiento completado!")

Se encontraron 298 imágenes en alas_png
Cargando DINOv2...


Using cache found in C:\Users\linet/.cache\torch\hub\facebookresearch_dinov2_main


Procesando 06274Q-2-A.png...
Guardado: particulas\06274Q-2-A.png


C:\Users\linet\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


Procesando 06274Q-3-A.png...


...

C:\Users\linet\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


Guardado: particulas\TAB03-2-A.png
Procesando VER99-2-A.png...


C:\Users\linet\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


Guardado: particulas\VER99-2-A.png
Procesando VER99-3-A.png...
Guardado: particulas\VER99-3-A.png
¡Procesamiento completado!


C:\Users\linet\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(
